In [1]:
import sys

sys.path.append("technique")
from utils import *

In [2]:
import pandas as pd

In [3]:
import culture.centre_choregraphique
import dee.activite
import dee.cantine
import mobilite.mobilite
import sports.sports

In [4]:
from openfisca_france import CountryTaxBenefitSystem

In [5]:
tbs = CountryTaxBenefitSystem()
tbs.load_extension("openfisca_france_local")

In [6]:
full = pd.concat(
    [
        culture.centre_choregraphique.get_results(tbs),
        dee.activite.get_results(tbs),
        dee.cantine.get_results(tbs),
        mobilite.mobilite.get_results(tbs),
        sports.sports.get_results(tbs),
    ],
    ignore_index=True,
)

In [7]:
pd.options.display.float_format = lambda v: "{:_.0f}".format(v).replace("_", " ")

In [8]:
full.sort_values("Recettes", ascending=False, ignore_index=True)

,Direction,Service,Recettes,Quantité
0,Mobilité,Plein tarif,13 300 925,598 224
1,Mobilité,Réduit,8 899 971,648 576
2,DEE,Repas standard,5 426 779,1 430 517
3,Sport,10 entrées,1 347 605,45 777
4,DEE,APM,949 262,24 470
5,Sport,entrée unitaire,850 442,245 342
6,Sport,abo_annuel,386 800,2 338
7,DEE,AL_J,269 534,36 016
8,Sport,patinoire entree unitaire,267 469,59 264
9,DEE,Repas végétarien,261 280,76 815


In [ ]:
from openfisca_core.parameters.parameter_scale import (
    ParameterScale,
    SingleAmountTaxScale,
)
from openfisca_core import parameters

In [7]:
baremes = {
    "apl_1_2j": lambda p: p.communes.strasbourg.periscolaire_loisirs.demi_journee.bareme,
    "apl_j": lambda p: p.communes.strasbourg.periscolaire_loisirs.journee.bareme,
    "apm": lambda p: p.communes.strasbourg.periscolaire_maternelle.bareme,
    "dee_std": lambda p: p.metropoles.strasbourg.tarifs_cantine,
    "dee_veg": lambda p: p.metropoles.strasbourg.tarifs_repas_vege,
    "dee_pan": lambda p: p.metropoles.strasbourg.tarifs_repas_panier,
    "ccs_eveil": lambda p: p.communes.strasbourg.centre_choregraphique.eveil.bareme,
    "ccs_enf_1c": lambda p: p.communes.strasbourg.centre_choregraphique.enfant._1_cours.bareme,
    "ccs_enf_2c": lambda p: p.communes.strasbourg.centre_choregraphique.enfant._2_cours.bareme,
    "ccs_enf_3c": lambda p: p.communes.strasbourg.centre_choregraphique.enfant._3_cours.bareme,
    "ccs_enf_4c": lambda p: p.communes.strasbourg.centre_choregraphique.enfant._4_cours.bareme,
    "ccs_adu_1c_tri": lambda p: p.communes.strasbourg.centre_choregraphique.adulte._1_cours_trimestre.bareme,
    "ccs_adu_1c": lambda p: p.communes.strasbourg.centre_choregraphique.adulte._1_cours.bareme,
    "ccs_adu_2c": lambda p: p.communes.strasbourg.centre_choregraphique.adulte._2_cours.bareme,
    "ccs_adu_3c": lambda p: p.communes.strasbourg.centre_choregraphique.adulte._3_cours.bareme,
    "ccs_adu_4c": lambda p: p.communes.strasbourg.centre_choregraphique.adulte._4_cours.bareme,
    "cons_enf12_agent": lambda p: p.communes.strasbourg.conservatoire.traditionnel.agent_ems.enfant_12,
    "cons_enf12_ems": lambda p: p.communes.strasbourg.conservatoire.traditionnel.habitant_ems.enfant_12,
    "cons_enf12_ext": lambda p: p.communes.strasbourg.conservatoire.traditionnel.hors_ems.enfant_12,
    "cons_enf3": lambda p: p.communes.strasbourg.conservatoire.traditionnel.enfant_3,
    "cons_ha_ems_1": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.habitant_ems.enfant_1,
    "cons_ha_ext_1": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.hors_ems.enfant_1,
    "cons_ha_ems_2": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.habitant_ems.enfant_2,
    "cons_ha_ext_2": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.hors_ems.enfant_2,
    "cons_ha_ems_3": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.habitant_ems.enfant_3,
    "cons_ha_ext_3": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.hors_ems.enfant_3,
    "cons_ha_ems_4": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.habitant_ems.enfant_4,
    "cons_ha_ext_4": lambda p: p.communes.strasbourg.conservatoire.horaires_amenages.hors_ems.enfant_4,
    "cts_base": lambda p: p.metropoles.strasbourg.tarification_solidaire.bareme_reduit,
    "cts_reduit": lambda p: p.metropoles.strasbourg.tarification_solidaire.bareme,
    "pat_pu": lambda p: p.communes.strasbourg.patinoire.entree_unitaire.bareme_qf,
    "pat_10": lambda p: p.communes.strasbourg.patinoire._10_entrees.bareme_qf,
    "pis_pu": lambda p: p.communes.strasbourg.piscine.entree_unitaire.bareme_qf,
    "pis_10": lambda p: p.communes.strasbourg.piscine._10_entrees.bareme_qf,
    "pis_abo_ann": lambda p: p.communes.strasbourg.piscine.abonnement_annuel,
    "pis_abo_ete": lambda p: p.communes.strasbourg.piscine.abonnement_ete,
    "pis_cycle": lambda p: p.communes.strasbourg.piscine.cycle.bareme,
    "pis_stage_ete": lambda p: p.communes.strasbourg.piscine.stage_ete.bareme,
    "pis_stage_vac": lambda p: p.communes.strasbourg.piscine.stage_vacances.bareme,
    "pis_stage_5s": lambda p: p.communes.strasbourg.piscine.stage_5_seances.bareme,
}
baremes_ = {
    "apl_1_2j": lambda p: p.communes.strasbourg.periscolaire_loisirs.demi_journee.bareme,
}

In [8]:
for b in baremes:
    assert type(baremes[b](tbs.parameters)) == ParameterScale

In [9]:
len(baremes)

40

In [10]:
p = tbs.get_parameters_at_instant(base_period)
threshold_rows = {}
for column in baremes:
    bareme = baremes[column](p)
    for index, level in enumerate(bareme.thresholds):
        if level not in threshold_rows:
            threshold_rows[level] = {"QF": level}
        threshold_rows[level][column] = bareme.amounts[index]

In [11]:
level_data = [*threshold_rows.keys()]
level_data.sort()

In [12]:
for b in baremes:
    assert type(baremes[b](p)) == SingleAmountTaxScale

In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import os
import requests
import json

In [15]:
GRIST_DOC_ID = os.getenv("GRIST_DOC_ID")
GRIST_API_KEY = os.getenv("GRIST_API_KEY")
GRIST_SERVER = os.getenv("GRIST_SERVER")

In [16]:
columns = [{"id": "QF", "fields": {"type": "Numeric"}}]
for b in baremes:
    columns.append({"id": b, "fields": {"type": "Numeric"}})

In [17]:
res_table = requests.post(
    f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables",
    data=json.dumps({"tables": [{"id": "Tableau QF", "columns": columns}]}),
    headers={
        "Authorization": f"Bearer {GRIST_API_KEY}",
        "Content-Type": "application/json",
    },
).json()
res_table

{'tables': [{'id': 'Tableau_QF2'}]}

In [18]:
table_id = res_table["tables"][0]["id"]
table_id

'Tableau_QF2'

In [19]:
records = [{"fields": threshold_rows[i]} for i in level_data]

In [20]:
res_table = requests.post(
    f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables/{table_id}/records",
    data=json.dumps({"records": records}),
    headers={
        "Authorization": f"Bearer {GRIST_API_KEY}",
        "Content-Type": "application/json",
    },
).json()
res_table

{'records': [{'id': 1},
  {'id': 2},
  {'id': 3},
  {'id': 4},
  {'id': 5},
  {'id': 6},
  {'id': 7},
  {'id': 8},
  {'id': 9},
  {'id': 10},
  {'id': 11},
  {'id': 12},
  {'id': 13},
  {'id': 14},
  {'id': 15},
  {'id': 16},
  {'id': 17},
  {'id': 18}]}

In [32]:
table_id = "Tableau_QF"
res_records = requests.get(
    f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables/{table_id}/records",
    headers={
        "Authorization": f"Bearer {GRIST_API_KEY}",
    },
).json()
res_records

{'records': [{'id': 1,
   'fields': {'apl_j': 2.84,
    'apl_1_2j': 2.13,
    'apm': 11.95,
    'dee_std': 1.5,
    'dee_veg': 1.15,
    'dee_pan': 0.75,
    'ccs_eveil': 165,
    'ccs_enf_1c': 921,
    'ccs_enf_2c': 241,
    'ccs_enf_3c': 314,
    'ccs_enf_4c': 380,
    'ccs_adu_1c_tri': 99,
    'ccs_adu_1c': 238,
    'ccs_adu_2c': 430,
    'ccs_adu_3c': 608,
    'ccs_adu_4c': 762,
    'cts_base': 3.4,
    'cts_reduit': 5.8,
    'pat_pu': 1,
    'pat_10': 35,
    'pis_pu': 1,
    'pis_10': 21,
    'pis_abo_ann': 120,
    'pis_abo_ete': 10,
    'pis_cycle': 15,
    'pis_stage_ete': 5,
    'pis_stage_vac': 10,
    'pis_stage_5s': 3,
    'QF': 0}},
  {'id': 2,
   'fields': {'apl_j': None,
    'apl_1_2j': None,
    'apm': None,
    'dee_std': None,
    'dee_veg': None,
    'dee_pan': None,
    'ccs_eveil': None,
    'ccs_enf_1c': None,
    'ccs_enf_2c': None,
    'ccs_enf_3c': None,
    'ccs_enf_4c': None,
    'ccs_adu_1c_tri': None,
    'ccs_adu_1c': None,
    'ccs_adu_2c': None,
    'cc

In [31]:
atbs = CountryTaxBenefitSystem()
atbs.load_extension("openfisca_france_local")

In [77]:
rtbs = CountryTaxBenefitSystem()
rtbs.load_extension("openfisca_france_local")

In [78]:
base_period
period = "2022-01"

In [79]:
for b in baremes:
    brackets = []
    for r in res_records["records"]:
        if b in r["fields"] and r["fields"][b] is not None:
            brackets.append(
                parameters.ParameterScaleBracket(
                    data={
                        "threshold": {period: {"value": r["fields"]["QF"]}},
                        "amount": {period: {"value": r["fields"][b]}},
                    }
                )
            )
    bb = baremes[b](rtbs.parameters)
    bb.brackets = brackets

In [80]:
df, compens_constant = mobilite.mobilite.get_df()

In [81]:
rdf = pd.DataFrame(
    data={
        "pu_fichier": df.PU,
        "pu_calc": df.PU * 0,
        "tp": df.PU * 0,
    }
)
data, complement_df = mobilite.mobilite.build_data(df, rdf)

In [82]:
scenario = StrasbourgSurveyScenario(rtbs, data=data)

In [83]:
variable_name = "eurometropole_strasbourg_tarification_transport"
variable = scenario.simulation.tax_benefit_system.get_variable(
    variable_name, check_existence=True
)

In [92]:
full_r = pd.concat(
    [
        culture.centre_choregraphique.get_results(rtbs),
        dee.activite.get_results(rtbs),
        dee.cantine.get_results(rtbs),
        mobilite.mobilite.get_results(rtbs),
        sports.sports.get_results(rtbs),
    ],
    ignore_index=True,
)

In [93]:
full_r

,SERVICE,RECETTES
0,ENF_CP_EI_AN,48_068
1,ADU_xx_1C_TR,99
2,ADU_xx_1C_AN,104_058
3,ADU_xx_2C_AN,53_597
4,ADU_xx_3C_AN,12_388
5,ADU_xx_4C_AN,4_289
6,ENF_CL_1C_AN,55_165
7,ENF_CL_2C_AN,83_241
8,ENF_CL_3C_AN,10_770
9,ENF_CL_4C_AN,12_734


In [95]:
comp_df = full.merge(full_r, on=["SERVICE"], suffixes=["", "_R"]).sort_values(
    "RECETTES", ascending=False
)

In [96]:
comp_df

,SERVICE,RECETTES,RECETTES_R
15,Repas standard,5_426_779,5_426_779
18,Transports,1_850_075,1_877_828
20,10 entrées,1_347_605,1_347_605
10,APM,949_262,949_262
19,entrée unitaire,850_442,850_442
21,abo_annuel,386_800,386_800
11,AL_J,269_534,269_534
28,patinoire entree unitaire,267_469,267_469
16,Repas végétarien,261_280,261_280
22,abo_annuel_ce,168_700,168_700
